In [1]:
!ls ../data/task_1_query_product_ranking/

product_catalogue-v0.3.csv        test_public-v0.3.csv
sample_submission_public-v0.3.csv train-v0.3.csv


In [39]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.getcwd()

'/Users/rdubey/mysrc/random-stuff/nlp-transformers'

In [40]:
import pandas as pd
dir = '/Users/rdubey/mysrc/random-stuff/data/task_1_query_product_ranking/'
train = pd.read_csv(dir+'train-v0.3.csv')
test = pd.read_csv(dir+'test_public-v0.3.csv')
submission = pd.read_csv(dir+'sample_submission_public-v0.3.csv')
product = pd.read_csv(dir+'product_catalogue-v0.3.csv')
len(train), len(test), len(submission), len(product)

(781744, 48696, 48696, 883868)

In [24]:
# filter for US
train = train[train.query_locale == 'us']
test = test[test.query_locale == 'us']
US_query_ids = set(train.query_id.tolist() + test.query_id.tolist())
submission = submission[submission.query_id.isin(US_query_ids)]
len(train), len(test), len(submission)

(419730, 26791, 26791)

In [25]:
train.head()

,query_id,query,query_locale,product_id,esci_label
0,0,# 2 pencils not sharpened,us,B0000AQO0O,exact
1,0,# 2 pencils not sharpened,us,B0002LCZV4,exact
2,0,# 2 pencils not sharpened,us,B00125Q75Y,exact
3,0,# 2 pencils not sharpened,us,B001AZ1D3C,exact
4,0,# 2 pencils not sharpened,us,B001B097KC,exact


In [26]:
test.head()

,query_id,query,query_locale,product_id
0,33779,#10 envelopes without security tint,us,B000DT87J4
1,33779,#10 envelopes without security tint,us,B000T9MP5U
2,33779,#10 envelopes without security tint,us,B0012ND8EY
3,33779,#10 envelopes without security tint,us,B001MBK5UG
4,33779,#10 envelopes without security tint,us,B007YX2KB8


In [27]:
submission.head()

,query_id,product_id
448,33779,B07R4TGNCF
449,33779,B08Y8VX63W
450,33779,B01FWMM738
451,33779,B07GJCSF8Z
452,33779,B085WJMWC9


In [41]:
product.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpene...",NaN,144 woodcase #2 HB pencils made from high-qual...,Amazon Basics,Yellow,us
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser,...",<p><strong>BACK TO BAZIC</strong></p><p>Our go...,&#11088; UN-SHARPENED #2 PREMIUM PENCILS. Each...,BAZIC Products,12-count,us
2,B07G7F6JZ6,Emraw Pre Sharpened Round Primary Size No 2 Ju...,<p><b>Emraw Pre-Sharpened #2 HB Wood Pencils -...,✓ PACK OF 8 NUMBER 2 PRESHARPENED BEGINNERS PE...,Emraw,Yellow,us
3,B07JZJLHCF,Emraw Pre Sharpened Triangular Primary Size No...,<p><b>Emraw Pre-Sharpened #2 HB Wood Pencils -...,✓ PACK OF 6 NUMBER 2 PRESHARPENED BEGINNERS PE...,Emraw,Yellow,us
4,B07MGKC3DD,"BIC Evolution Cased Pencil, #2 Lead, Gray Barr...",NaN,Premium #2 HB lead pencils with break-resistan...,Design House,Gray,us



    shopping_queries_dataset_examples.parquet contains the following columns : example_id, query, query_id, product_id, product_locale, esci_label, small_version, large_version, split
    shopping_queries_dataset_products.parquet contains the following columns : product_id, product_title, product_description, product_bullet_point, product_brand, product_color, product_locale
    shopping_queries_dataset_sources.csv contains the following columns : query_id, source


In [42]:
df_product_examples =  pd.merge(
    train,
    product,
    how='left',
    left_on=['query_locale', 'product_id'],
    right_on=['product_locale', 'product_id']
)

In [43]:
df_product_examples.head()

,query_id,query,query_locale,product_id,esci_label,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale
0,0,# 2 pencils not sharpened,us,B0000AQO0O,exact,"Ticonderoga Beginner Pencils, Wood-Cased #2 HB...",NaN,Round wood pencil with latex-free eraser\nFini...,Ticonderoga,Yellow,us
1,0,# 2 pencils not sharpened,us,B0002LCZV4,exact,"TICONDEROGA Tri-Write Triangular Pencils, Stan...",NaN,Triangular shape to promote proper grip\nExclu...,Ticonderoga,Yellow,us
2,0,# 2 pencils not sharpened,us,B00125Q75Y,exact,"TICONDEROGA Pencils, Wood-Cased, Unsharpened, ...",NaN,Exclusive #2 graphite formula provides extra s...,Ticonderoga,Yellow,us
3,0,# 2 pencils not sharpened,us,B001AZ1D3C,exact,"Ticonderoga Pencils, Wood-Cased Graphite #2 HB...",NaN,Exclusive #2 graphite formula provides extra s...,Ticonderoga,Yellow,us
4,0,# 2 pencils not sharpened,us,B001B097KC,exact,"Ticonderoga Laddie Pencils, Wood-Cased #2 HB S...",NaN,"Intermediate size diameter (11/32"") creates a ...",Ticonderoga,Yellow,us


In [35]:
retrieval_counts = train.groupby('query_id').agg(num_items=('product_id', 'count'))

In [36]:
retrieval_counts.num_items.value_counts()

16     12069
40      3414
39       537
15       449
14       216
       ...  
87         1
94         1
188        1
65         1
83         1
Name: num_items, Length: 69, dtype: int64

In [37]:
retrieval_counts.num_items.describe()

count    18849.000000
mean        22.268025
std         10.763524
min          8.000000
25%         16.000000
50%         16.000000
75%         34.000000
max        188.000000
Name: num_items, dtype: float64

In [12]:
train.esci_label.value_counts()

exact         341170
substitute    267963
irrelevant    132057
complement     40554
Name: esci_label, dtype: int64

In [13]:
len(train)

781744

In [38]:
train[train.query_id == 0]

,query_id,query,query_locale,product_id,esci_label
0,0,# 2 pencils not sharpened,us,B0000AQO0O,exact
1,0,# 2 pencils not sharpened,us,B0002LCZV4,exact
2,0,# 2 pencils not sharpened,us,B00125Q75Y,exact
3,0,# 2 pencils not sharpened,us,B001AZ1D3C,exact
4,0,# 2 pencils not sharpened,us,B001B097KC,exact
5,0,# 2 pencils not sharpened,us,B003JFL1WY,exact
6,0,# 2 pencils not sharpened,us,B004X4KRPM,substitute
7,0,# 2 pencils not sharpened,us,B004X4KRW0,exact
8,0,# 2 pencils not sharpened,us,B00DZB6SIE,irrelevant
9,0,# 2 pencils not sharpened,us,B00OFNI9VK,exact
